# Understanding Tokenization, Temperature, Top_P and Chat API

#### Install dependencies

In [1]:
%pip install --upgrade openai tiktoken wandb -qq
%pip install --upgrade wandb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Import libraries

In [2]:
import os
import openai
import tiktoken
import wandb
from pprint import pprint
from getpass import getpass
from wandb.integration.openai import autolog

#### Loading OPENAI API key

In [3]:
if os.getenv('OPENAI_API_KEY') is None:
    if any(['VSCODE' in x for x in os.environ.keys()]):
        print('Plase enter password in the VS Code prompt at the top of your VS Code window!')
    os.environ['OPENAI_API_KEY'] == getpass('Paste your OpenAI key from: https://paltform.openai.com/account/api-key\n')

assert os.getenv('OPENAI_API_KEY',''.startswith('sk-')), "This doesn't look like a valid OpenAI API key"
print('OpenAI API key configured')

OpenAI API key configured


#### Enable W&B to track our experiment

In [4]:
# start logging to W&B
os.environ['WANDB_NOTEBOOK_NAME'] = 'Using_APIs.ipynb'
# autolog(init={'project':'llmapps', 'job_type': 'introduction'})
wandb.init(project='llmapp', job_type='introduction')

wandb: Currently logged in as: mary1378. Use `wandb login --relogin` to force relogin


## Tokenization
for more information about tokenization you can [check this link](https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken)

In [5]:
MODEL = 'gpt-3.5-turbo-instruct'

In [6]:
encoding = tiktoken.encoding_for_model(MODEL)
enc = encoding.encode('Weights & Biases is awesome!')
print(enc)
print(encoding.decode(enc))

[56730, 612, 12371, 2315, 374, 12738, 0]
Weights & Biases is awesome!


we can decode the tokens one by one

In [7]:
for token_id in enc:
    print(f'{token_id}\t{encoding.decode([token_id])}')

56730	Weights
612	 &
12371	 Bi
2315	ases
374	 is
12738	 awesome
0	!


## Sampling
Lets sample some text from the model. For this, let's create a wrapper function around the temperature parameters. Higher temperature will result in more random samples

In [8]:
def generate_with_temperature(temp):
    'Generate text with a given temperature, higher tmperature means more randomness'
    response = openai.completions.create(
        model=MODEL,
        prompt='Say something about Weights & Biases',
        max_tokens=50,
        temperature=temp
    )
    return response.choices[0].text

In [9]:
for temp in [0, 0.5, 1, 1.5, 2]:
    pprint(f'TEMP: {temp}, GENERATION: {generate_with_temperature(temp)}')

('TEMP: 0, GENERATION: \n'
 '\n'
 'Weights & Biases is a machine learning platform that helps data scientists '
 'and machine learning engineers track, visualize, and collaborate on their '
 'experiments. It offers a suite of tools for experiment tracking, '
 'hyperparameter optimization, and model visualization, making it easier for')
('TEMP: 0.5, GENERATION: \n'
 '\n'
 'Weights & Biases is a machine learning platform that helps data scientists '
 'and machine learning engineers track and visualize their experiments, '
 'collaborate with team members, and deploy their models into production. It '
 'provides a centralized dashboard to monitor and compare different models, as '
 'well')
('TEMP: 1, GENERATION: \n'
 '\n'
 'Weights & Biases is a machine learning platform that allows data scientists '
 'and machine learning engineers to track and visualize their experiments, '
 'collaborate with team members, and deploy models into production. It offers '
 'powerful tools for hyperparameter

You can also use the top_p_parameter to control the diversity of the generated text. This parameter controls the cumulative probability of the next token. for example, if top_p=0.9 the model will pick the next token from the top 90% most likely tokens. The higher the top_p the more likely the model will pick a token that it hasn't seen before. You should use one of temperature or top_p at a given time.

In [10]:
def generate_with_topp(topp):
    'Generate text with a given top-p, higher top-p means more randomness'
    response = openai.completions.create(
        model=MODEL,
        prompt='Say something about Weights & Biases',
        max_tokens=50,
        top_p=topp
    )
    return response.choices[0].text

In [11]:
for topp in [0.01, 0.1, 0.5, 1]:
    pprint(f'TOP_P: {topp}, GENERATION: {generate_with_topp(topp)}')

('TOP_P: 0.01, GENERATION: \n'
 '\n'
 'Weights & Biases is a machine learning platform that helps data scientists '
 'and machine learning engineers track, visualize, and collaborate on their '
 'experiments. It offers a suite of tools for experiment tracking, '
 'hyperparameter optimization, and model visualization, making it easier for')
('TOP_P: 0.1, GENERATION: \n'
 '\n'
 'Weights & Biases is a machine learning platform that helps data scientists '
 'and machine learning engineers track, visualize, and collaborate on their '
 'experiments. It offers a suite of tools for experiment tracking, '
 'hyperparameter optimization, and model visualization, making it easier for')
('TOP_P: 0.5, GENERATION: \n'
 '\n'
 'Weights & Biases is a machine learning platform that helps data scientists '
 'and machine learning engineers track, visualize, and optimize their models. '
 'It offers a suite of tools for experiment tracking, hyperparameter tuning, '
 'and model debugging, making it easier to 

## Chat API

Let's switch to chat mode and see how the model responds to our message. We have some control over the model's response by passing a system-role, here we can steer to model to adhere to a certain behaviour. you can see [OpenAI docs here](https://platform.openai.com/docs/guides/chat-completions/overview) for more information about each role.

we are using gpt-3.5-turbu, this model is faster and cheaper than davinci-003

In [12]:
response = openai.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': 'Say something about Weights & Biases'}
    ],
    temperature=0
)
response

ChatCompletion(id='chatcmpl-9r6jeNrJ5SriDsn0GSOTm42bIF674', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Weights & Biases is a popular machine learning platform that helps researchers and data scientists track and visualize their machine learning experiments. It provides tools for experiment tracking, visualization, and collaboration, making it easier to manage and optimize machine learning projects.', role='assistant', function_call=None, tool_calls=None))], created=1722444302, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=47, prompt_tokens=26, total_tokens=73))

In [13]:
print(response.choices[0].message.content)

Weights & Biases is a popular machine learning platform that helps researchers and data scientists track and visualize their machine learning experiments. It provides tools for experiment tracking, visualization, and collaboration, making it easier to manage and optimize machine learning projects.


In [14]:
wandb.finish()